In [1]:
from tensorflow.keras import Sequential, layers

In [4]:
model = Sequential([
    layers.Input(shape=(80,)),
    layers.Embedding(input_dim=10000,output_dim=32), #->10000*32 파라미터
    layers.SimpleRNN(64),
    layers.Dense(2,activation='softmax')
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 80, 32)         │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 64)             │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 326,338 (1.24 MB)

 Trainable params: 326,338 (1.24 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
from tensorflow.keras.datasets import imdb
(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words=10000)
#0이 부정, 1이 긍정
X_train.shape,y_train.shape
X_train[0] #-> 이미 전처리된 상태
#-> # 실제로는 각 숫자가 단어 하나를 의미
y_train[0]

1

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
#-> pad_sequences는 길이를 맞춰주는 함수
#-> 리뷰마다 길이가 다 다르기 때문에, 딥러닝에 넣기 전에 길이를 통일해야 함
X_train_pad = pad_sequences(X_train,maxlen=80,truncating='post',padding='post')
X_test_pad = pad_sequences(X_test,maxlen=80,truncating='post',padding='post')

In [10]:
%%time
model.fit(X_train_pad,y_train,epochs=3,batch_size=200)

Epoch 1/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 288s 2s/step - accuracy: 0.6166 - loss: 0.6463
Epoch 2/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 332s 3s/step - accuracy: 0.8374 - loss: 0.3872
Epoch 3/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 344s 3s/step - accuracy: 0.9048 - loss: 0.2499
CPU times: user 27min 43s, sys: 46min 38s, total: 1h 14min 22s
Wall time: 16min 3s


In [12]:
model.evaluate(X_test_pad,y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 168s 215ms/step - accuracy: 0.7777 - loss: 0.5080


[0.5099249482154846, 0.7771599888801575]

In [13]:
import numpy as np
pred = model.predict(X_test_pad)
pred = np.argmax(pred, axis=1)

782/782 ━━━━━━━━━━━━━━━━━━━━ 243s 310ms/step


In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[ 9279,  3221],
       [ 2350, 10150]])

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.77716